In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F

In [2]:
import pandas as pd
full_corpus_path = "../data/full_corpus/full_corpus.csv"

def read_to_list(path):
    data_df = pd.read_csv(path)
    return data_df['text'].to_list()

fc = read_to_list(full_corpus_path)

print(len(fc))
print(fc[:4])

817480
['[\'Najmłodsi obchody swojego święta w tym roku rozpoczęli w sobotę. Odbyło się wiele imprez dla najmłodszych - festynów, zabaw. Dopisała pogoda. Sponsorzy lokalnych imprez zapewnili dzieciom słodycze i zabawę. Sobotni festyn na ul. Monte Cassino w Sopocie, zorganizowany przez Fundację Rodzina Nadziei, w tym roku nosił nazwę "Ulica Walecznych Serc".\']', "['Konserwator zabytków ustalił szkody, jakie w jednej z komnat Zamku Królewskiego na Wawelu spowodował nietrzeźwy turysta z województwa suwalskiego. Mężczyzna wskoczył do łoża Zygmunta Starego i uszkodził przykrywającą je kapę. Zdarzenie miało miejsce w minioną sobotę. 36-letni Robert J. w towarzystwie kilkuosobowej grupy znajomych zwiedzał Zamek Królewski na Wawelu.']", "['– musi wyjść..', '– a na razie to samochodzik bym chciał jednak kupić bo to to takie wiesz kurde no w sumie mógłbym sobie kupić czinkłeczento na przykład nie? jeździ..', '– znaczy nawet mógłbyś sobie kupić za tą kwotę sejczento i to nie stare bo półtora roc

In [7]:
def clean_text(text: str) -> str:
    text = text.replace('[', '')
    text = text.replace(']', '')
    return text.replace('\'', '')

fc = [clean_text(line) for line in fc]

In [9]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ndazhunts/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
from nltk.tokenize import word_tokenize

fc_tokenized = [word_tokenize(text=line, language='polish') for line in fc]
fc_tokenized[:4]

[['Najmłodsi',
  'obchody',
  'swojego',
  'święta',
  'w',
  'tym',
  'roku',
  'rozpoczęli',
  'w',
  'sobotę',
  '.',
  'Odbyło',
  'się',
  'wiele',
  'imprez',
  'dla',
  'najmłodszych',
  '-',
  'festynów',
  ',',
  'zabaw',
  '.',
  'Dopisała',
  'pogoda',
  '.',
  'Sponsorzy',
  'lokalnych',
  'imprez',
  'zapewnili',
  'dzieciom',
  'słodycze',
  'i',
  'zabawę',
  '.',
  'Sobotni',
  'festyn',
  'na',
  'ul.',
  'Monte',
  'Cassino',
  'w',
  'Sopocie',
  ',',
  'zorganizowany',
  'przez',
  'Fundację',
  'Rodzina',
  'Nadziei',
  ',',
  'w',
  'tym',
  'roku',
  'nosił',
  'nazwę',
  '``',
  'Ulica',
  'Walecznych',
  'Serc',
  "''",
  '.'],
 ['Konserwator',
  'zabytków',
  'ustalił',
  'szkody',
  ',',
  'jakie',
  'w',
  'jednej',
  'z',
  'komnat',
  'Zamku',
  'Królewskiego',
  'na',
  'Wawelu',
  'spowodował',
  'nietrzeźwy',
  'turysta',
  'z',
  'województwa',
  'suwalskiego',
  '.',
  'Mężczyzna',
  'wskoczył',
  'do',
  'łoża',
  'Zygmunta',
  'Starego',
  'i',
  'u

In [11]:
from typing import List

def flatten(nested_list: List[List[str]]) -> List[str]:
    return [item for sublist in nested_list for item in sublist]

vocab = list(set(flatten(fc_tokenized)))
vocab[:4]

['Azikiwe', 'RedHawks', 'Boitsov', 'Siarnicki']

In [13]:
len(vocab)

2163500

In [14]:
from typing import Dict

def get_word2idx(vocabulary: List[str]) -> Dict[str, int]:
    return {w: idx for (idx, w) in enumerate(vocabulary)}

def get_idx2word(vocabulary: List[str]) -> Dict[int, str]:
    return {idx: w for (idx, w) in enumerate(vocabulary)}

In [15]:
def get_idx_pairs(tokenized_corpus: List[List[str]]) -> np.ndarray:
    window_size = 2
    idx_pairs = []
    word2idx = get_word2idx(vocabulary=vocab)
    # for each sentence
    for sentence in tokenized_corpus:
        indices = [word2idx[word] for word in sentence]
        # for each word, treated as center word
        for center_word_pos in range(len(indices)):
            # for each window position
            for w in range(-window_size, window_size + 1):
                context_word_pos = center_word_pos + w
                # make sure not to jump out sentence
                if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                    continue
                context_word_idx = indices[context_word_pos]
                idx_pairs.append((indices[center_word_pos], context_word_idx))

    return np.array(idx_pairs)

In [16]:
def get_input_layer(word_idx):
    x = torch.zeros(vocab).float()
    x[word_idx] = 1.0
    return x

In [17]:
def train_embeddings(vocabulary_size: int, idx_pairs: np.ndarray) -> None:
    embedding_dims = 5
    W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
    W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
    num_epochs = 101
    learning_rate = 0.001

    for epo in range(num_epochs):
        loss_val = 0
        for data, target in idx_pairs:
            x = Variable(get_input_layer(data)).float()
            y_true = Variable(torch.from_numpy(np.array([target])).long())

            z1 = torch.matmul(W1, x)
            z2 = torch.matmul(W2, z1)

            log_softmax = F.log_softmax(z2, dim=0)

            loss = F.nll_loss(log_softmax.view(1,-1), y_true)
            loss_val += loss.data[0]
            loss.backward()
            W1.data -= learning_rate * W1.grad.data
            W2.data -= learning_rate * W2.grad.data

            W1.grad.data.zero_()
            W2.grad.data.zero_()
        if epo % 10 == 0:
            print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')


In [ ]:
fc_idx_pairs = get_idx_pairs(tokenized_corpus=fc_tokenized)
vocab_size = len(vocab)

In [ ]:
train_embeddings(vocabulary_size=vocab_size, idx_pairs=fc_idx_pairs)